# Reconhecimento de entidades nomeada 

Reconhece as entidades nomeadas dos documentos do dataset.

**Entrada**: "`dataset.zip`" e "`datasetpos.zip`"

**Saída**: "`datasetner.zip`"

**Processamento**:
1. Copia e descompacta o arquivo "`dataset.zip`" e "`datasetpos.zip`" para a máquina local do 
Google Colab.
2. Realiza o reconhecimento das entidades nomeadas dos documentos do conjunto de dados utilizando a ferramenta de PLN spaCy. 
3. Gera o arquivo "`datasetner.csv`" com os dados das PoS-Tagging.
4. Compacta o arquivo "`datasetner.csv`" para "`datasetner.zip`"
5. Copia o arquivo "`datasetner.zip`" para o google drive.

**Estrutura do arquivo `datasetner.csv` compactado em `datasetner.zip`.**

Cada linha do arquivo `datasetpos.csv` é formado por `["id","ner_documento"]`.
 - `"id"` é o idenficador do documento no dataset.
 - `"ner_documento"` é uma lista com as entidades reconhecidas do documento, formado por `["indice_sentenca","lista_entidades"]`.
  - `"indice_sentenca"` é o índice da sentença no documento.
  - `"lista_entidades"` é uma lista com as entidades reconhecidas na sentença formado por `["texto entidade"`,`"tipo entidade"`, `"posição inicial"` e `"posição final"]`.


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [1]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório do notebook no diretório corrente.   


In [2]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioNotebook():
    """
      Verifica se existe o diretório do notebook no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_NOTEBOOK):  
        # Cria o diretório
        os.makedirs(DIRETORIO_NOTEBOOK)
        logging.info("Diretório do notebook criado: {}".format(DIRETORIO_NOTEBOOK))
    
    return DIRETORIO_NOTEBOOK

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [3]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempoArredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempoArredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [4]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModelosParametros:
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )

Biblioteca de limpeza de tela


In [5]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [6]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [7]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [8]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [9]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def getDeviceGPU():
    """
    Retorna um dispositivo de GPU se disponível ou CPU.
    
    Retorno:
    `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [10]:
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Conecta o modelo ao device

In [11]:
# Import das bibliotecas.
import torch # Biblioteca para manipular os tensores

def conectaGPU(model, device):
    """
      Conecta um modelo BERT a GPU.

      Parâmetros:
        `model` - Um modelo BERT carregado.       
        `device` - Um device de GPU.     
    
      Retorno:
        `model` - Um objeto model BERT conectado a GPU.     
    """
    # Associa a GPU ao modelo.
    model.to(device)

    # Se existe GPU disponível.
    if torch.cuda.is_available():    
        # Diga ao pytorch para rodar este modelo na GPU.
        logging.info("Pytorch rodando o modelo na GPU.")
        model.cuda()
        
    else:
        logging.info("Pytorch rodando sem GPU.")

    return model

Memória

Memória disponível no ambiente

In [12]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [13]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")     

Mounted at /content/drive


## 1.7 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [14]:
# Instala dependências do spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [15]:
# Instala uma versão específica
!pip install -U spacy==3.4.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.4.4 which is incompatible.


# 2 Parametrização

## Gerais

In [16]:
# Definição dos parâmetros a serem avaliados

## Específicos

Parâmetros do modelo

In [17]:
# Definição dos parâmetros do Modelo.
model_args = ModelosParametros(     

    #modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",
    modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",
        
)

## Nome do diretório dos arquivos de dados

In [18]:
# Diretório do notebook
DIRETORIO_NOTEBOOK = "SRI"

## Define o caminho para os arquivos de dados

In [19]:
# Diretório local para os arquivos de dados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_NOTEBOOK + "/"

# Diretório no google drive com os arquivos de dados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/" + DIRETORIO_NOTEBOOK + "/data/"

# 3 spaCy

## 3.1 Download arquivo modelo

Uso:
https://spacy.io/usage

Modelos:
https://spacy.io/models

In [20]:
!python -m spacy download $model_args.modelo_spacy

2023-04-25 18:52:31.405245: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')


## 3.2 Carrega o modelo

In [21]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

nlp = spacy.load(model_args.modelo_spacy)

# 4 Reconhecendo entidades nomeadas

## 4.1 Carregamento do Dataset e PoS-Tagging

### 4.1.1 Especifica os nomes dos arquivos de dados



In [22]:
# Nome do arquivo
NOME_ARQUIVO_DATASET = "dataset.csv"
NOME_ARQUIVO_DATASET_COMPACTADO = "dataset.zip"
NOME_ARQUIVO_DATASET_POS = "datasetpos.csv"
NOME_ARQUIVO_DATASET_POS_COMPACTADO = "datasetpos.zip"

### 4.1.2 Cria o diretório local para receber os dados

In [23]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

INFO:root:Diretório criado: /content/SRI


### 4.1.3 Copia e descompacta os arquivos do Google Drive para o Colaboratory

In [24]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_DATASET_COMPACTADO" "$DIRETORIO_LOCAL"
  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_DATASET_POS_COMPACTADO" "$DIRETORIO_LOCAL"

  logging.info("Terminei a cópia.")

INFO:root:Terminei a cópia.


Descompacta os arquivos.

Usa o unzip para descompactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 
*   `-d` Diretório de destino


In [25]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_COMPACTADO" -d "$DIRETORIO_LOCAL"
  !unzip -o -j -q "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_POS_COMPACTADO" -d "$DIRETORIO_LOCAL"

  logging.info("Terminei a descompactação.")

INFO:root:Terminei a descompactação.


### 4.1.4 Carregamento das lista com os dados dos arquivos e postagging

#### Carrega o arquivo dos dados e POS

In [26]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_dataset = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET, sep=";", encoding="UTF-8")
df_dataset_pos = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET_POS, sep=";", encoding="UTF-8")

logging.info("TERMINADO DOCUMENTOS: {}.".format(len(df_dataset)))
logging.info("TERMINADO DOCUMENTOS POS: {}.".format(len(df_dataset_pos)))

INFO:root:TERMINADO DOCUMENTOS: 388.
INFO:root:TERMINADO DOCUMENTOS POS: 388.


In [27]:
df_dataset.sample(5)

,id,sentencas,documento
274,686,"['Oriento medidas de isolamento', 'até 07/05 O...",Oriento medidas de isolamento até 07/05 Orient...
174,452,"['PACIENTE SEM GRAVIDADE.', 'SOLICITO TESTE PC...",PACIENTE SEM GRAVIDADE. SOLICITO TESTE PCR. FO...
213,532,['Solicito RT-PCR para Covid-19; Prescrevo sin...,Solicito RT-PCR para Covid-19; Prescrevo sinto...
254,647,"['PACIENTE SEM GRAVIDADE.', 'ENVIO PEDIDO DE R...",PACIENTE SEM GRAVIDADE. ENVIO PEDIDO DE RT-PCR...
94,246,"['Ajusto medicação, acrescento Prednisona, Lor...","Ajusto medicação, acrescento Prednisona, Lorat..."


In [28]:
df_dataset_pos.sample(5)

,id,pos_documento
332,869,"[[['Prescrevo', 'sintomáticos', ';'], ['PROPN'..."
286,716,"[[['Conduta', ':', 'Realizo', 'AM', 'de', '1',..."
164,428,"[[['PACIENTE', 'SEM', 'GRAVIDADE', '.'], ['NOU..."
345,896,"[[['SOLICITO', ':', 'SOROLOGIA', 'P/', 'DENGUE..."
256,649,"[[['Oriento', 'retorno', 'ao', 'trabalho', 'se..."


#### Corrigir os tipos de colunas dos dados e POS

Em lista documento:
- coluna 1 - `sentenças` carregadas do arquivo vem como string e não como lista.

Em lista pos:
- coluna 1 - `pos_documento` carregadas do arquivo vem como string e não como lista.

In [29]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

# Verifica se o tipo da coluna não é list e converte
df_dataset["sentencas"] = df_dataset["sentencas"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

df_dataset_pos["pos_documento"] = df_dataset_pos["pos_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO DOCUMENTOS: {}.".format(len(df_dataset)))
logging.info("TERMINADO CORREÇÃO DOCUMENTOS POS: {}.".format(len(df_dataset_pos)))

INFO:root:TERMINADO CORREÇÃO DOCUMENTOS: 388.
INFO:root:TERMINADO CORREÇÃO DOCUMENTOS POS: 388.


#### Criando dados indexados 

In [30]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
df_dataset_indexado = df_dataset.set_index(["id"])
df_dataset_indexado.head()

,sentencas,documento
id,,
1,"[Solicito RT-PCR, SARSCOV 2., Forneço atestado...",Solicito RT-PCR SARSCOV 2. Forneço atestado mé...
2,"[Conduta: Realizo AM de 1 dia (s), Prescrevo s...",Conduta: Realizo AM de 1 dia (s) Prescrevo sin...
9,"[Forneço atestado médico 08 dias., Isolamento ...",Forneço atestado médico 08 dias. Isolamento so...
10,"[Solicito RT-PCR, SARSCOV 2., Forneço atestado...",Solicito RT-PCR SARSCOV 2. Forneço atestado mé...
13,"[Conduta: Realizo AM de 1 dia (s), Prescrevo s...",Conduta: Realizo AM de 1 dia (s) Prescrevo sin...


In [31]:
# Expecifica o(s) campo(s) indexado(s) e faz uma cópia da lista indexada
df_dataset_pos_indexado = df_dataset_pos.set_index(["id"])
df_dataset_pos_indexado.head()

,pos_documento
id,
1,"[[[Solicito, RT-PCR], [PROPN, VERB], [RT-PCR],..."
2,"[[[Conduta, :, Realizo, AM, de, 1, dia, (, s, ..."
9,"[[[Forneço, atestado, médico, 08, dias, .], [A..."
10,"[[[Solicito, RT-PCR], [PROPN, VERB], [RT-PCR],..."
13,"[[[Conduta, :, Realizo, AM, de, 1, dia, (, s, ..."


## 4.2 Reconhendo as entidades nomeadas

O SpaCy reconhece os seguintes tipos de entidades:
- PERSON - Pessoas, inclusive fictícias.
- NORP - Nacionalidades ou grupos religiosos ou políticos.
- FAC - Edifícios, aeroportos, rodovias, pontes, etc.
- ORG - Empresas, agências, instituições, etc.
- GPE - Países, cidades, estados.
- LOC - Locais não GPE, cordilheiras, corpos d'água.
- PRODUCT - Objetos, veículos, alimentos, etc. (Não são serviços.)
- EVENT - Furacões nomeados, batalhas, guerras, eventos esportivos, etc.
- WORK_OF_ART - Títulos de livros, músicas, etc.
- LAW - Documentos nomeados transformados em leis.
- LANGUAGE  - Qualquer idioma nomeado.
- DATE - Datas ou períodos absolutos ou relativos.
- TIME - Tempos menores que um dia.
- PERCENT - Porcentagem, incluindo "%".
- MONEY  - Valores monetários, incluindo a unidade.
- QUANTITY  - Medidas, como peso ou distância.
- ORDINAL - "primeiro", "segundo", etc.
- CARDINAL  - Numerais que não se enquadram em outro tipo.

### 4.2.1 Reconhecendo as entidades nomeadas

In [32]:
# Import das bibliotecas.
import re # Biblioteca para expressões regulares
import os # Biblioteca para acessar o sistema de arquivos
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

logging.info("Processando {} documentos.".format(len(df_dataset)))

# Barra de progresso dos dados
dados_bar = tqdm_notebook(df_dataset.iterrows(), desc=f"Dados", unit=f"registro", total=len(df_dataset))

# Lista para armazenar as entidades reconhecidas do documento
lista_dataset_ner = []

# Percorre as linhas dos documentos
for i, linha_documento in dados_bar:

    # Carrega a lista das sentenças do documento de acordo com o tipo armazenado
    lista_sentenca_documento = linha_documento[1]

    # Lista para armazenar as entidades das sentenças do documento   
    lista_sentenca_documentos_ner = []

    # Percorre as sentenças do documento
    for j, sentenca in enumerate(lista_sentenca_documento):
      #print(sentenca)

      # Processa sentença no spaCy para extrair as entidades
      doc = nlp(sentenca)

      # Armazena as entidades da sentença
      lista_ner_sentenca = []
      for ent in doc.ents:
          #print(ent.text, ent.label_, ent.start_char, ent.end_char)
          # Guarda o texto da entidade, o tipo da entidade, a posição inicial e a posição final
          lista_ner_sentenca.append([ent.text, ent.label_, ent.start_char, ent.end_char])

      # Concatena o pos do documento
      lista_sentenca_documentos_ner.append([j, lista_ner_sentenca])
      
    # Adiciona o documento a lista
    lista_dataset_ner.append([linha_documento[0], lista_sentenca_documentos_ner])  

INFO:root:Processando 388 documentos.


Dados:   0%|          | 0/388 [00:00<?, ?registro/s]

## 4.3 Salva as Entidades geradas

Gera o arquivo com as entidades reconhecidas e depois compacta o arquivo para enviar para o Google Drive.


### 4.3.1 Especifica os nomes dos arquivos das entidades do dataset


In [33]:
# Nome do arquivo
NOME_ARQUIVO_DATASET_NER = "datasetner.csv"
NOME_ARQUIVO_DATASET_NER_COMPACTADO = "datasetner.zip"

### 4.3.2 Gera arquivo das entidades

In [34]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_dataset_ner = pd.DataFrame(lista_dataset_ner, columns = ["id","ner_documento"])

# Salva o arquivo das postagging
df_dataset_ner.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET_NER,  sep=";", index=False)

### 4.3.3 Compacta e copia a entidades reconhecidas do dataset para uma pasta do GoogleDrive

Compacta o arquivo gerado da comparação para facilitar o envio para o GoogleDrive

In [35]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_NER_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_NER"

  logging.info("Terminei a compactação.")

INFO:root:Terminei a compactação.


Copia o arquivo para o GoogleDrive

In [36]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
   
    # Copia o arquivo das postagging       
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_DATASET_NER_COMPACTADO" "$DIRETORIO_DRIVE"
        
    logging.info("Terminei a cópia do arquivo.")

INFO:root:Terminei a cópia do arquivo.


### 4.3.4 Carrega os dados

Realiza um teste carregando o arquivo da entidades do dataset criado.


In [37]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_dataset_ner = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_DATASET_NER, sep=";", encoding="UTF-8")

print(len(df_dataset_pos))

388


Corrigir o tipo de dados da lista das entidades

Na lista  pos:
- coluna 1 - `ner_documento` carregadas do arquivo vem como string e não como lista.

In [38]:
# Import das bibliotecas.
import ast # Biblioteca para conversão de string em lista

df_dataset_ner["ner_documento"] = df_dataset_ner["ner_documento"].apply(lambda x: ast.literal_eval(x) if type(x)!=list else x)

logging.info("TERMINADO CORREÇÃO DOCUMENTOS NER: {}.".format(len(df_dataset_ner)))

INFO:root:TERMINADO CORREÇÃO DOCUMENTOS NER: 388.


In [39]:
df_dataset_ner.sample(5)

,id,ner_documento
331,868,"[[0, [['Oriento', 'LOC', 0, 7]]]]"
354,908,"[[0, [['CD= DECONGEX PLUS', 'ORG', 0, 17]]], [..."
41,95,"[[0, []], [1, []], [2, []]]"
73,185,"[[0, []], [1, [['SN Manter sintomáticos', 'MIS..."
212,531,"[[0, [['Orientações', 'LOC', 0, 11]]], [1, []]..."


## 4.4 Exemplo

Exemplo gráfico do reconhecimento de entidade em um documento.

### Texto a ser analisado

In [58]:
# Recupera o primeiro documento
documento = df_dataset['documento'][4]

# Ou especifique diretamente um texto
#documento = "O Brasil, um vasto país sul-americano, estende-se da Bacia Amazônica, no norte, até os vinhedos e as gigantescas Cataratas do Iguaçu, no sul. O Rio de Janeiro, simbolizado pela sua estátua de 38 metros de altura do Cristo Redentor, situada no topo do Corcovado, é famoso pelas movimentadas praias de Copacabana e Ipanema, bem como pelo imenso e animado Carnaval, com desfiles de carros alegóricos, fantasias extravagantes e samba."

# Mostra o documento
print(documento)

Conduta: Realizo AM de 1 dia (s) Prescrevo sintomáticos SN (Dipirona, Cetoprofeno, Zyrtec, Avamys, Bromexina, Lavagem nasal com SF); Oriento ainda a retornar o contato em caso de piora dos sintomas; Deixo o serviço disponível em caso de duvidas ou necessidade de novas orientações


### Armazena um texto.

In [59]:
# Submete o texto ao spaCy
doc = nlp(documento)

logging.info('Texto armazenado!')

INFO:root:Texto armazenado!


### Identificando as entidades. (NER/Named Entity Recognition)

**Text**: o texto original da entidade.<br>
**Start**: Índice de início da entidade no Doc.<br>
**End**: Índice do fim da entidade no Doc.<br>
**Label**: rótulo da entidade, ou seja, tipo.<br>

https://medium.com/botsbrasil/como-reconhecer-entidades-na-l%C3%ADngua-portuguesa-usando-spacy-8a5ca6f42c4f

### Listando as entidades identificadas

In [60]:
for ent in doc.ents:
   print(ent.text, ent.start_char, ent.end_char, ent.label_)

Prescrevo sintomáticos SN 33 58 MISC
Cetoprofeno 70 81 PER
Zyrtec 83 89 PER
Lavagem nasal 110 123 MISC
Oriento 133 140 LOC


ou

In [61]:
[(entity, entity.label_) for entity in doc.ents]

[(Prescrevo sintomáticos SN, 'MISC'),
 (Cetoprofeno, 'PER'),
 (Zyrtec, 'PER'),
 (Lavagem nasal, 'MISC'),
 (Oriento, 'LOC')]

### Visualizando o texto marcado

In [62]:
# Importando as bibliotecas.
import spacy 
from spacy  import displacy

displacy.render(doc,style="ent",jupyter=True)

Sobre as entidades reconhecidas:
  Os tipos "PER" e "MISC" são classificados como:
  
    "PER": "Named person or family."
    "MISC": "Miscellaneous entities, e.g. events, nationalities, products or works of art"

**Font**: https://github.com/explosion/spaCy/blob/master/spacy/glossary.py

# 5 Finalização

## 5.1 Tempo final de processamento



In [51]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:05:17 (h:mm:ss)
